<a href="https://colab.research.google.com/github/AliciaFalconCaro/LLM_Chatbot_Movies/blob/main/DataAnalysisForMoviesChatBotLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data downloaded from:
https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows/data?select=imdb_top_1000.csv


In [1]:
import pandas as pd
df = pd.read_csv("./local_dir/imdb_top_1000.csv")
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [2]:
# Let's first explore the data.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


The majority of the data information is non-numeric/text (categorical)
There is a total of 1000 entries.

In [4]:
#Let's check null values:
print("Missing values:", df.isnull().sum())

Missing values: Poster_Link        0
Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64


The features 'certificate', 'meta_score' and 'Gross' contains a high number of null values. In this case, for simplicity, we drop all NaN values (missing values).

In [5]:
df_cleaned = df.dropna()
print("Missing values:", df_cleaned.isnull().sum())

Missing values: Poster_Link      0
Series_Title     0
Released_Year    0
Certificate      0
Runtime          0
Genre            0
IMDB_Rating      0
Overview         0
Meta_score       0
Director         0
Star1            0
Star2            0
Star3            0
Star4            0
No_of_Votes      0
Gross            0
dtype: int64


We use a freely available LLM for this example. In this case, we use DialoGPT

We need to create question-answers pairs based on the available dataset.

In [7]:
# Function to generate question-answer pairs
def generate_qa_pairs(row):
    qa_pairs = []

    # Example questions for each column
    if "Series_Title" in row and "Genre" in row:
        qa_pairs.append({
            "instruction": f"What is the genre of {row['Series_Title']}?",
            "response": row["Genre"]
        })
    if "Series_Title" in row and "IMDB_Rating" in row:
        qa_pairs.append({
            "instruction": f"What is the IMDb rating of {row['Series_Title']}?",
            "response": str(row["IMDB_Rating"])  # Convert numeric values to string
        })
    if "Series_Title" in row and "Gross" in row:
        qa_pairs.append({
            "instruction": f"How much did {row['Series_Title']} earn?",
            "response": str(row["Gross"]) if pd.notna(row["Gross"]) else "Unknown"
        })
    if "Released_Year" in row:
        qa_pairs.append({
            "instruction": f"In what year was {row['Series_Title']} released?",
            "response": str(row["Released_Year"])
        })
    if "Series_Title" in row and "Certificate" in row:
        qa_pairs.append({
            "instruction": f"What is the certificate for {row['Series_Title']}?",
            "response": row["Certificate"]
        })
    if "Series_Title" in row and "Runtime" in row:
        qa_pairs.append({
            "instruction": f"How long is {row['Series_Title']}?",
            "response": str(row["Runtime"])
        })
    if "Series_Title" in row and "Overview" in row:
        qa_pairs.append({
            "instruction": f"What is the overview of {row['Series_Title']}?",
            "response": row["Overview"]
        })
    if "Series_Title" in row and "Meta_score" in row:
        qa_pairs.append({
            "instruction": f"What is the meta score of {row['Series_Title']}?",
            "response": str(row["Meta_score"])
        })
    if "Series_Title" in row and "Director" in row:
        qa_pairs.append({
            "instruction": f"Who is the director of {row['Series_Title']}?",
            "response": row["Director"]
        })

    return qa_pairs

# Iterate over rows to create question-answer pairs
qa_data = []
for _, row in df_cleaned.iterrows():
    qa_data.extend(generate_qa_pairs(row))

# Display a few examples
print(qa_data[:5])


[{'instruction': 'What is the genre of The Shawshank Redemption?', 'response': 'Drama'}, {'instruction': 'What is the IMDb rating of The Shawshank Redemption?', 'response': '9.3'}, {'instruction': 'How much did The Shawshank Redemption earn?', 'response': '28,341,469'}, {'instruction': 'In what year was The Shawshank Redemption released?', 'response': '1994'}, {'instruction': 'What is the certificate for The Shawshank Redemption?', 'response': 'A'}]


We save the questions-answers pairs as json file to pass it to the model to train it.

In [8]:
import json

# Save question-answer pairs to a JSON file
output_path = "qa_data.json"
with open(output_path, "w") as f:
    json.dump(qa_data, f, indent=4)

print(f"Question-answer pairs saved to {output_path}")


Question-answer pairs saved to qa_data.json


We load the json file

In [9]:
# Load the JSON file containing question-answer pairs
input_path = "qa_data.json"  # Replace with your actual file path
with open(input_path, "r") as f:
    qa_data = json.load(f)

print(f"Loaded {len(qa_data)} question-answer pairs.")


Loaded 6426 question-answer pairs.


Some LLM requires specific fields in the training json file for the fine-tuning of the model.

In this case, we will use GPT2, which requires:


"instruction": qa["instruction"],


"context": "",  # Optionally, include a context if needed


"response": qa["response"]

In [10]:
# Modify/reformat data to include additional fields
formatted_data = []

for qa in qa_data:
    formatted_entry = {
        "instruction": qa["instruction"],  # Keep the original question
        "context": "",  # Optionally, include a context if needed
        "response": qa["response"]  # Keep the original answer
    }
    formatted_data.append(formatted_entry)

print(f"Formatted {len(formatted_data)} question-answer pairs.")


Formatted 6426 question-answer pairs.


In [11]:
#we save the reformated json file
output_path = "formatted_qa_data.json"  # Specify your desired output path
with open(output_path, "w") as f:
    json.dump(formatted_data, f, indent=4)

print(f"Formatted question-answer pairs saved to {output_path}")


Formatted question-answer pairs saved to formatted_qa_data.json


Given any of the two json files containing the question-answer pairs, we load the data as a huggingface dataset.

In [12]:
!pip install transformers datasets accelerate

In [13]:
from datasets import Dataset

# Load the JSON data
with open("formatted_qa_data.json", "r") as f:
    qa_data = json.load(f)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({
    "text": [
        f"Question: {entry['instruction']} Context: {entry['context']} Answer: {entry['response']}"
        for entry in qa_data
    ]
})


We load GPT2 model and tokenizer

In [14]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split dataset into train and test if not already present
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Add labels to the tokenized dataset
def add_labels(examples):
    examples["labels"] = examples["input_ids"]  # labels should be same as input_ids
    return examples

# Apply label addition
train_dataset = train_dataset.map(add_labels, batched=True)
test_dataset = test_dataset.map(add_labels, batched=True)


Map:   0%|          | 0/6426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5783 [00:00<?, ? examples/s]

Map:   0%|          | 0/643 [00:00<?, ? examples/s]

We use Hugging Face’s Trainer API for training of the model

In [16]:
from transformers import Trainer, TrainingArguments

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=10,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)


Now we fine-tune the model and save the trained model

To use the model, you need the API key. It can be obtained from: https://wandb.ai/

In [25]:
#This step could take some time
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.023400,0.031179
2,0.020400,0.032258
3,0.018600,0.033890


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=4338, training_loss=0.02088342365882209, metrics={'train_runtime': 757.578, 'train_samples_per_second': 22.901, 'train_steps_per_second': 5.726, 'total_flos': 9066310926336000.0, 'train_loss': 0.02088342365882209, 'epoch': 3.0})

In [26]:
trainer.save_model("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")

#we save the tokenizer to ensure that the same tokens are used for the model next time. They can be imported/loaded as:
#model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
#tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")



('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

Now we test the fine-tuned model

In [31]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")

# Create a text generation pipeline
qa_pipeline = pipeline("text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Test with a new question
prompt = "Question: What is the genre of The Godfather? Context: Answer:"
result = qa_pipeline(prompt, max_length=50, num_return_sequences=1)
print(result[0]["generated_text"])


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Question: What is the genre of The Godfather? Context: Answer: Crime, Thriller


We verify the correct output of the chatbot through a query to the dataset.

In [29]:
print("The correct Genre is:",df_cleaned[df_cleaned["Series_Title"] == "The Godfather"]["Genre"])

The correct Genre is: 1    Crime, Drama
Name: Genre, dtype: object


Now we create the interactive chatbot

In [ ]:
#this is the chatbot with the fine-tuned model
def chatbot_response(input_text):
    prompt = f"Question: {input_text}? Context: Answer:"
    result = qa_pipeline(prompt, max_length=50, num_return_sequences=1)
    return result[0]["generated_text"]

#start chatbot:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print(f"Bot: {response}")

User: Question: What is the genre of The Godfather?
Bot: Question: Question: What is the genre of The Godfather?? Context: Answer: Action, Comedy, Family
User: Question: What is the genre of The Godfather?
Bot: Question: Question: What is the genre of The Godfather?? Context: Answer: Crime, Drama
